<a href="https://colab.research.google.com/github/pdh0184/Celine_project/blob/main/Resnet50%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9C%A0%EC%82%AC_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%B0%BE%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-gpu

In [ ]:
import pandas as pd
import base64
from PIL import Image
from io import BytesIO
import numpy as np
import cv2
import matplotlib.pyplot as plt
import faiss
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from google.colab.patches import cv2_imshow  # Colab에서 cv2.imshow() 대신 사용

df = pd.read_csv('/content/drive/MyDrive/개인 프로젝트 파일/Celine_project/CELINE_DATA.csv')
df.head()

In [ ]:
df.shape

### 이미지 디코딩
- Base64 데이터 디코딩 후 img_list에 추가


In [ ]:
img_list = []

for index, row in df.iterrows():

  image_str64 = row['이미지 데이터']
  try:
    image_data = base64.b64decode(image_str64)
    img_out = Image.open(BytesIO(image_data))
    img_list.append(img_out)

  except Exception as e:
    print(f"Error decoding image data for index {index}: {e}")
    img_list.append(None)
    continue



### 비슷한 이미지 조회
- FAISS 사용
  - 모든 벡터를 메모리에 저장하고, 검색 시 전체 벡터를 비교하는 방식
  - 단순하고 직관적이지만, 데이터가 많을 때 속도가 느려질 수 있다
  - 작은 데이터셋에서는 매우 효율적입니다.

## ResNet 가져오기
- 이미지의 특징을 추출해야하기 때문에 마지막 층은 제거하고 사용한다

In [ ]:
import faiss
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from PIL import Image

# 사전 훈련된 모델 로드 (ResNet50 사용) , 가장 빠르고 성능도 좋음

# Pre-trained ResNet 모델 불러오기
model = models.resnet50(pretrained=True)

# ResNet50에서 마지막 classification layer를 제거합니다.
modules = list(model.children())[:-1]
model = nn.Sequential(*modules)

# 학습된 모델을 evaluation mode로 설정합니다.
model.eval()


In [ ]:

# 이미지 전처리 함수
def preprocess_image(image_data):
    img = Image.open(image_data)
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    img_tensor = transform(img).unsqueeze(0)
    print(img_tensor.shape)
    return img_tensor


In [ ]:
# 이미지 특징 추출 함수
def extract_features(image_data):
    img_tensor = preprocess_image(image_data)
    with torch.no_grad():
        features = model(img_tensor)
    return features.numpy().flatten()


In [ ]:
# 모든 이미지에 대해 특징 벡터 추출
zero_array = np.zeros(2048)
features_list = []

for index, row in df.iterrows():
    image_str64 = row['이미지 데이터']
    try:
        image_data = BytesIO(base64.b64decode(image_str64))
        features = np.array(extract_features(image_data))
        print(features.shape)
        features_list.append(features)
    except Exception as e:
        print(f"Error decoding image data for index {index}: {e}")
        features_list.append(zero_array)



In [ ]:
# 특징 벡터 리스트를 NumPy 배열로 변환
features_array = np.array(features_list, dtype=np.float32)
print(features_array[0].shape)
print(features_array[160].shape)

In [ ]:
features_array.shape

In [ ]:
# FAISS 인덱스 생성 (L2 거리 사용)
index = faiss.IndexFlatL2(features_array.shape[1])
print('차원 수 :',index.d)
print('현재 인덱스에 추가된 벡터의 총 개수:',index.ntotal)

In [ ]:
# 특징 벡터를 인덱스에 추가
index.add(features_array)
print(index.ntotal)

### FAISS 에서 이미지 간 유사도 계산으로 유사 제품 검색

In [ ]:
# 이미지 간 유사도 계산
D, I = index.search(features_array, k=11)  # 각 이미지에 대해 가장 유사한 10개 검색
print("Distances:", D)
print("Indices:", I)


In [ ]:
input_product = int(input("제품 번호 입력") )#원하는 이미지 선택 시 해당 되는 인덱스 번호가 있다고 가정

start_time = time.time()

i = 0

for index in I[input_product]:
  if i == 0: #처음 인덱스는 자기 자신이므로 제외한다
    i += 1
    continue
  print(df["제품명"].iloc[index])
  print(df["가격"].iloc[index])
  print(df["상세주소"].iloc[index])

  plt.imshow(img_list[index])
  plt.axis('off')  # 축 제거
  plt.show()
  i += 1

end_time = time.time()

# 소요 시간 계산
elapsed_time = end_time - start_time
print(f"총 연산 시간: {elapsed_time:.6f} seconds")
print(f"총 추천 제품 개수: {i-1}개")

